In [110]:
import oracledb
import folium
import geojson

from shapely.wkt import dumps, loads

# Establishing and testing connection

In [111]:
cs = oracledb.makedsn("dbmanage.lab.ii.agh.edu.pl", 1521, sid="DBMANAGE")

un = "student"
pw = "stu638dent"

connection = oracledb.connect(user=un, password=pw, dsn=cs)

In [112]:
print(connection)

<oracledb.Connection to student@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=dbmanage.lab.ii.agh.edu.pl)(PORT=1521))(CONNECT_DATA=(SID=DBMANAGE)))>


# Choosing schema US_SPAT

In [113]:
q = """ALTER SESSION SET CURRENT_SCHEMA = US_SPAT"""

cursor = connection.cursor()

cursor.execute(q)

In [114]:
cursor = connection.cursor()

q = """select id, state from us_states""";

for row in cursor.execute(q):
    print(row)

(29, 'Nevada')
(30, 'New Hampshire')
(31, 'New Jersey')
(32, 'New Mexico')
(8, 'Delaware')
(9, 'District of Columbia')
(10, 'Florida')
(11, 'Georgia')
(12, 'Hawaii')
(13, 'Idaho')
(14, 'Illinois')
(21, 'Maryland')
(22, 'Massachusetts')
(23, 'Michigan')
(24, 'Minnesota')
(25, 'Mississippi')
(26, 'Missouri')
(27, 'Montana')
(28, 'Nebraska')
(1, 'Alabama')
(2, 'Alaska')
(3, 'Arizona')
(4, 'Arkansas')
(5, 'California')
(6, 'Colorado')
(7, 'Connecticut')
(15, 'Indiana')
(16, 'Iowa')
(17, 'Kansas')
(18, 'Kentucky')
(19, 'Louisiana')
(20, 'Maine')
(33, 'New York')
(34, 'North Carolina')
(35, 'North Dakota')
(36, 'Ohio')
(37, 'Oklahoma')
(38, 'Oregon')
(39, 'Pennsylvania')
(40, 'Rhode Island')
(41, 'South Carolina')
(42, 'South Dakota')
(43, 'Tennessee')
(44, 'Texas')
(45, 'Utah')
(46, 'Vermont')
(47, 'Virginia')
(48, 'Washington')
(49, 'West Virginia')
(50, 'Wisconsin')
(51, 'Wyoming')
(52, 'American Samoa')
(53, 'Guam')
(54, 'Northern Mariana Islands')
(55, 'Puerto Rico')
(56, 'Virgin Island

In [115]:
def OutputTypeHandler(cursor, name, defaultType, size, precision, scale):
    if defaultType == oracledb.CLOB:
        return cursor.var(oracledb.LONG_STRING, arraysize = cursor.arraysize)
    
connection.outputtypehandler = OutputTypeHandler  

# Example 1 - Selecting landlocked countries

In [116]:
m = folium.Map()

q = """SELECT  sdo_util.to_wktgeometry(geometry)
       FROM world_countries 
       WHERE landlocked = 'Y'"""

r = loads(cursor.execute(q).fetchall())

st = {'fillColor': 'yellow', 'color': 'orange'}

l = []

for row in r:
    g = geojson.Feature(geometry=row[0], properties={})
    l.append(g)

feature_collection = geojson.FeatureCollection(l)
folium.GeoJson(feature_collection, style_function=lambda x:st).add_to(m)  

m 

# Example 2 - all states that have something common with Mississippi river

In [117]:
m = folium.Map()

q = """SELECT  sdo_util.to_wktgeometry(s.geom)
       FROM us_rivers r, us_states s
       WHERE r.name = 'Mississippi'
       and SDO_ANYINTERACT (s.geom, r.geom) ='TRUE'"""

r = loads(cursor.execute(q).fetchall())

st = {'fillColor': 'pink', 'color': 'magenta'}

l = []
for row in r:
    g = geojson.Feature(geometry=row[0], properties={})
    l.append(g)


feature_collection = geojson.FeatureCollection(l)
folium.GeoJson(feature_collection, style_function=lambda x:st).add_to(m)  

m

In [118]:
q = """SELECT  sdo_util.to_wktgeometry(geom)
       FROM us_rivers where name = 'Mississippi'"""

r = loads(cursor.execute(q).fetchall())

st2 = {'color': 'blue'}

l = []

for row in r:
    g = geojson.Feature(geometry=row[0], properties={})
    l.append(g)


feature_collection = geojson.FeatureCollection(l)

folium.GeoJson(feature_collection, style_function=lambda x:st2).add_to(m)  

m 

# Example 3 - National parks located only inside in Arizona

In [129]:
m = folium.Map()

q = """SELECT  sdo_util.to_wktgeometry(p.geom)
FROM us_parks p, us_states s
WHERE s.state = 'Arizona'
AND SDO_INSIDE (p.geom, s.geom ) = 'TRUE'"""

r = loads(cursor.execute(q).fetchall())

st = {'fillColor': 'blue', 'color': 'red'}

l = []
for row in r:
    g = geojson.Feature(geometry=row[0], properties={})
    l.append(g)


feature_collection = geojson.FeatureCollection(l)
folium.GeoJson(feature_collection, style_function=lambda x:st).add_to(m)  

m